In [ ]:
#! pip install gcsfs

In [ ]:
#! pip install koreanize_matplotlib

In [ ]:
#!pip install seaborn

In [2]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
import os

# GCS 파일 경로에서 데이터 불러오기 위한 라이브러리
from google.cloud import storage
from google.oauth2 import service_account

In [3]:
# Pandas 보기 옵션
# pd.set_option('display.max_columns',100)
# pd.set_option('display.max_rows',100)

In [4]:
# test
# # GCP 키 경로
# key_path = "../config/codeit_project_vm_key.json"
# # GCS 파일 경로
# gcs_path = "gs://part4_project/hackle/hackle_events.parquet"
# # 데이터 불러오기
# events_df = pd.read_parquet(gcs_path, storage_options={"token": None})

# # 확인
# display(events_df.head())
# events_df.shape

In [5]:
# 인증
key_path = "../config/codeit_project_vm_key.json"
credentials = service_account.Credentials.from_service_account_file(key_path)

# GCS 클라이언트 생성
client = storage.Client(credentials=credentials)
bucket = client.bucket("part4_project")

# 'hackle/' 경로에 있는 .parquet 파일 리스트 가져오기
blobs = bucket.list_blobs(prefix="hackle/")
hackle_file_list = [f"gs://part4_project/{blob.name}" for blob in blobs if blob.name.endswith(".parquet")]

# 결과 확인
hackle_file_list

['gs://part4_project/hackle/device_properties.parquet',
 'gs://part4_project/hackle/hackle_events.parquet',
 'gs://part4_project/hackle/hackle_properties.parquet',
 'gs://part4_project/hackle/user_properties.parquet']

In [6]:
# 파일들을 각각 변수로 불러오기
for title in hackle_file_list:
    # GCS 경로에서 파일명 추출
    file_name = title.split('/')[-1].replace('.parquet', '')
    
    # Parquet 파일 읽기
    df = pd.read_parquet(title, storage_options={"token": key_path})
    
    # 각각의 이름_df로 저장 (ex. device_properties_df)
    globals()[f"{file_name}_df"] = df
    
    # 선택적으로 출력 확인
    print(f"{file_name}_df 불러오기 완료, shape: {df.shape}")


device_properties_df 불러오기 완료, shape: (252380, 4)
hackle_events_df 불러오기 완료, shape: (11441319, 11)
hackle_properties_df 불러오기 완료, shape: (525350, 8)
user_properties_df 불러오기 완료, shape: (230819, 5)


In [27]:

# 기본 정보 확인 함수 
def show_df_info(df, df_name):
    """
    DataFrame의 기본 정보와 통계 요약을 출력하는 함수
    """
    print(f"\n{df_name} 형태:")
    display(df.shape)
    print(f"\n{df_name} 첫 5행:")
    display(df.head())
    print(f"\n{df_name} 정보:")
    df.info()
    print(f"\n{df_name} 통계 요약:")
    display(df.describe())
    print(f"\n{df_name} 통계 요약:")
    display(df.describe(include = 'O'))
    print(f"\n{df_name} 결측치 확인:")
    display(df.isnull().sum())
    # print(f"\n{df_name} 중복값 확인:")
    # print(df.iloc[:,1:].duplicated().sum())

# 1. hackle_properties

|테이블명|	컬럼명	|컬럼 설명|
|-------|----------|--------|
|hackle_properties|	session_id|	세션 식별자|
|		|user_id|	사용자 ID|
|		|language|	사용 언어|
|		|osname|	운영 체제 이름|
|		|osversion|	운영 체제 버전|
|		|versionname|	앱 버전 이름|
|		|device_id|	장치 ID|

In [28]:
show_df_info(hackle_properties_df, 'hackle_properties_df')


hackle_properties_df 형태:


(525350, 8)


hackle_properties_df 첫 5행:


,id,session_id,user_id,language,osname,osversion,versionname,device_id
0,1,4OzYh3seq3VKytpSn5pvQkZNQii1,,ko-KR,iOS,16.0,2.0.0,590E7C79-CBA0-44D8-8BE3-3C9BFABBBC74
1,2,8QXy31PQxbW9qLzq0Y1dhR8Ypm52,1046711,ko-KR,iOS,16.5.1,2.0.3,D5417226-F71B-4A9E-A180-CD072F2AB279
2,3,6bcea65d-9f40-46fc-888c-700fe707483f,1545130,ko,Android,13,2.0.5,6bcea65d-9f40-46fc-888c-700fe707483f
3,4,XVYNT6zfhFWqIg9omwg2AHDjTLx2,1224793,ko,Android,13,2.0.5,a05c1595-3e05-434b-8684-218b528bd725
4,5,XFB2SPiGfjbVhvJ3Q3DBsaT3m2B3,1329450,ko-US,iOS,16.5.1,2.0.5,EAC6C0B3-7CE8-40EA-8A91-9977C0BA5EF3



hackle_properties_df 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525350 entries, 0 to 525349
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           525350 non-null  int64 
 1   session_id   525350 non-null  object
 2   user_id      525350 non-null  object
 3   language     525350 non-null  object
 4   osname       525350 non-null  object
 5   osversion    525350 non-null  object
 6   versionname  525350 non-null  object
 7   device_id    525350 non-null  object
dtypes: int64(1), object(7)
memory usage: 32.1+ MB

hackle_properties_df 통계 요약:


,id
count,525350.000000
mean,262675.500000
std,151655.626297
min,1.000000
25%,131338.250000
50%,262675.500000
75%,394012.750000
max,525350.000000



hackle_properties_df 통계 요약:


,session_id,user_id,language,osname,osversion,versionname,device_id
count,525350,525350,525350,525350,525350,525350,525350
unique,253616,327381,151,2,74,16,251720
top,040914e1-61ac-40ef-b76a-718066d880dc,,ko-KR,iOS,16.5.1,2.0.5,040914e1-61ac-40ef-b76a-718066d880dc
freq,9,82255,340900,359479,218699,309644,9



hackle_properties_df 결측치 확인:


id             0
session_id     0
user_id        0
language       0
osname         0
osversion      0
versionname    0
device_id      0
dtype: int64

In [ ]:
# id 컬럼 제외하고 중복 확인 - 없음
hackle_properties_df.iloc[:,1:].duplicated().sum()

0

In [39]:
# user_id 가 공백??
hackle_properties_df[hackle_properties_df['user_id'] == ''].shape

(82255, 8)

- user_id가 공백으로 입력되어 있는 경우 82255 개 
- 해당 경우 'unknown' 으로 변경

In [51]:
# user_id에 공백만 입력된 경우 'unknown'으로 변경
hackle_properties_df['user_id'] = hackle_properties_df['user_id'].apply(lambda x: 'unknown' if isinstance(x, str) and x.strip() == '' else x)
hackle_properties_df['user_id'].value_counts()

user_id
unknown    82255
1578652       17
1459833       13
1571506       13
1239584       12
           ...  
1085617        1
1499469        1
1075954        1
1021052        1
1043127        1
Name: count, Length: 327381, dtype: int64

- 공백만 unknown으로 변경된 것 확인! (개수 동일하게 82255개)

In [64]:
# user_id마다 session_id는 몇 개? (hackle_events에서 session_id를 user_id와 동일 취급 가능한가?)
hackle_properties_df.groupby('user_id')['session_id'].nunique().reset_index(name='session_id_count').sort_values(by='session_id_count', ascending=False)


,user_id,session_id_count
319303,unknown,78197
158764,1459833,13
195272,1578652,13
192533,1571506,13
210229,855879,12
...,...,...
109620,1302497,1
109619,1302488,1
109618,1302487,1
109617,1302485,1


- user_id 형태가 다른 숫자형(으로 보이지만 object)과는 다르게 zzzYYwN42NdbhcyXHgyeAxyNH7J2 와 같은게 있음...!

In [ ]:
# 샘플 아무거나 확인 
hackle_properties_df[hackle_properties_df['user_id'].apply(len) > 7]

,id,session_id,user_id,language,osname,osversion,versionname,device_id
5,6,LztzUUFoRxdqTSPgQrX3MAAyNkM2,LztzUUFoRxdqTSPgQrX3MAAyNkM2,ko-KR,iOS,16.1,2.0.5,3F199073-9390-4137-B0B0-0DC4FC103009
27,28,qLdDlFGK9qObRuGXK20KAGbqzRZ2,qLdDlFGK9qObRuGXK20KAGbqzRZ2,ko,Android,13,2.0.3,7ffeeed2-6f34-45d9-82b1-9119c453135b
46,47,98078b29-9edc-4097-bb7a-500141473bb3,xQeEzeBtbBSX7gsxJCWFJ3XvXU02,ko,Android,12,2.0.0,98078b29-9edc-4097-bb7a-500141473bb3
78,79,ijJOxCkzhyZ0DJaQYq0JAbCnURO2,ijJOxCkzhyZ0DJaQYq0JAbCnURO2,ko-KR,iOS,16.5.1,2.0.5,E386A7FB-6FE5-4D00-BBDA-7B0E9E3DCE93
84,85,3nBBDbvgWaNeqPvsE4YnCQT9ILt2,3nBBDbvgWaNeqPvsE4YnCQT9ILt2,en-KR,iOS,16.5.1,2.0.3,DABCC8B8-2541-499B-BA8D-9CCCFBFF41A4
...,...,...,...,...,...,...,...,...
525279,525280,wllWQhzLEjeuvyAlXqK7m5Gy47O2,wllWQhzLEjeuvyAlXqK7m5Gy47O2,ko-KR,iOS,16.2,2.0.0,CB3FFDE9-3D58-4E9C-9E57-4AFA3FD22B2D
525284,525285,PImMsU1ENshcEnaEPpF9PyTOsWn1,PImMsU1ENshcEnaEPpF9PyTOsWn1,ko-KR,iOS,16.6,2.0.0,564E8019-0AD4-4E8B-967C-7F61C3952493
525287,525288,Nc4yxR3OfcVC1lDWv12vIiITovW2,Nc4yxR3OfcVC1lDWv12vIiITovW2,ko,Android,13,1.2.16,48e9f708-9db3-47ef-a11c-38aba1e72473
525335,525336,cpZ6XwYLTkbwDHpBkjp9bPLjVE53,cpZ6XwYLTkbwDHpBkjp9bPLjVE53,ko-KR,iOS,16.0,1.2.15,D0776AFE-BC11-43E5-9A68-535B352E5E63


- user_id가 눈으로 보기에도 찐 문자열인 것들이 있음 
- session_id와 user_id가 같은 경우도 있음

In [ ]:
# 한 user_id에 session_id 갯수 13개인 것 샘플 확인 
hackle_properties_df[hackle_properties_df['user_id'] == '1459833']

,id,session_id,user_id,language,osname,osversion,versionname,device_id
2255,2256,E7291C7D-54D9-459B-B93D-EB3DB0B5EB54,1459833,ko-KR,iOS,16.3.1,2.0.5,E7291C7D-54D9-459B-B93D-EB3DB0B5EB54
24115,24116,EF35B630-C9C9-4CEF-B70A-C5EB2703C17F,1459833,ko-KR,iOS,16.3.1,2.0.5,EF35B630-C9C9-4CEF-B70A-C5EB2703C17F
30217,30218,3CF3AF75-ECF6-4A07-A4F3-66EE0D114428,1459833,ko-KR,iOS,16.3.1,2.0.5,3CF3AF75-ECF6-4A07-A4F3-66EE0D114428
61146,61147,91AC528E-4CE9-4E3E-9283-98DD3F0E40C1,1459833,ko-KR,iOS,16.3.1,2.0.5,91AC528E-4CE9-4E3E-9283-98DD3F0E40C1
64455,64456,4BC78F41-A9F2-400E-AEC6-1C9449163BC2,1459833,ko-KR,iOS,16.3.1,2.0.3,4BC78F41-A9F2-400E-AEC6-1C9449163BC2
106580,106581,09BE6E5A-527E-4B50-AD7F-AC1680BD569D,1459833,ko-KR,iOS,16.3.1,2.0.5,09BE6E5A-527E-4B50-AD7F-AC1680BD569D
109773,109774,44A0482C-3668-432A-A367-6545D06EE753,1459833,ko-KR,iOS,16.3.1,2.0.5,44A0482C-3668-432A-A367-6545D06EE753
129211,129212,9C58E582-94AB-40E4-A77F-6044F43322E1,1459833,ko-KR,iOS,16.3.1,2.0.5,9C58E582-94AB-40E4-A77F-6044F43322E1
160205,160206,09D832EA-FAEF-460D-9612-62B8795EC54F,1459833,ko-KR,iOS,16.3.1,2.0.5,09D832EA-FAEF-460D-9612-62B8795EC54F
162316,162317,7033EC1F-1684-4887-8204-092EBEC2253A,1459833,ko-KR,iOS,16.3.1,2.0.3,7033EC1F-1684-4887-8204-092EBEC2253A


- 뭐지...? 아까 화면 공유로 봤을 때 user_id에 session_id도 다 1개씩 이었던 거 같은데?
- 그렇다면 hackle_events 테이블에서 session_id를 user_id로 봐도 되는게 맞나..?

# 2. device_properties

| # |      테이블명     |     컬럼명    |  컬럼 설명  |
|:-:|:-----------------:|:-------------:|:-----------:|
| 2 | device_properties | device_id     | 장치 ID     |
|   |                   | device_model  | 장치 모델명 |
|   |                   | device_vendor | 장치 제조사 |

In [82]:
show_df_info(device_properties_df, 'device_properties_df')


device_properties_df 형태:


(252380, 4)


device_properties_df 첫 5행:


,id,device_id,device_model,device_vendor
0,1,000007C9-E103-4EB5-9777-A9084D4952DF,"iPhone14,7",Apple
1,2,00002245-458F-4CDD-8533-B448CD43DBD2,"iPhone14,7",Apple
2,3,00012620-313A-4502-9F8D-8DAB7443215B,"iPhone14,5",Apple
3,4,000137bc-80de-4bb5-b61d-df7f217a4501,SM-F711N,samsung
4,5,000227D6-B782-4367-91C4-486B76DF9E37,"iPhone12,3",Apple



device_properties_df 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252380 entries, 0 to 252379
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             252380 non-null  int64 
 1   device_id      252380 non-null  object
 2   device_model   252380 non-null  object
 3   device_vendor  252380 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.7+ MB

device_properties_df 통계 요약:


,id
count,252380.000000
mean,126190.500000
std,72855.974806
min,1.000000
25%,63095.750000
50%,126190.500000
75%,189285.250000
max,252380.000000



device_properties_df 통계 요약:


,device_id,device_model,device_vendor
count,252380,252380,252380
unique,251720,522,36
top,5dab2e2f-54fd-4c4d-9996-1cccfa6c7428,"iPhone14,5",Apple
freq,2,27656,174444



device_properties_df 결측치 확인:


id               0
device_id        0
device_model     0
device_vendor    0
dtype: int64

In [83]:
# id 컬럼 제외하고 중복 확인 - 없음
device_properties_df.iloc[:,1:].duplicated().sum()

0

In [ ]:
# 제조사별 갯수 확인 
device_properties_df['device_vendor'].value_counts()

device_vendor
Apple            174444
samsung           75887
LGE                1578
Xiaomi              284
OnePlus              49
Google               31
LENOVO               24
Chinoe               19
FYD                   7
motorola              6
Lenovo                5
Nothing               4
Samsung               4
HUAWEI                3
Amazon                3
ZTE                   3
vivo                  3
blackshark            3
Sony                  2
APEX                  2
Teclast               2
incar                 2
asus                  2
Shanling              1
E-dong                1
IMUZ                  1
HXY                   1
Razer                 1
realme                1
Alldocube             1
Teclast_korea         1
bnd                   1
Digital               1
vsmart                1
Foxconn               1
iMUZ                  1
Name: count, dtype: int64

- Apple이 제일 많음
- 그 다음 samsung, Samsung 
- 제조사가 'nothing' 인 경우 4개 있음 

# 3. hackle_events

| # |    테이블명   |     컬럼명     |        컬럼 설명        |
|:-:|:-------------:|:--------------:|:-----------------------:|
| 3 | hackle_events | event_id       | 고유 이벤트 ID          |
|   |               | event_datetime | 이벤트 발생 시간        |
|   |               | event_key      | 이벤트 키               |
|   |               | session_id     | 세션 ID                 |
|   |               | item_name      | 이벤트 관련 아이템 이름 |
|   |               | page_name      | 이벤트 발생 페이지 이름 |
|   |               | friend_count   | 친구 수                 |
|   |               | votes_count    | 받은 투표 수            |
|   |               | heart_balance  | 잔여 하트               |
|   |               | question_id    | 고유 질문 ID            |

In [85]:
show_df_info(hackle_events_df, 'hackle_events_df')


hackle_events_df 형태:


(11441319, 11)


hackle_events_df 첫 5행:


,event_id,event_datetime,event_key,session_id,id,item_name,page_name,friend_count,votes_count,heart_balance,question_id
0,00000533-3f1c-4b3b-81f1-0c8f35754b4e,2023-07-18 19:40:17,$session_start,4OzYh3seq3VKytpSn5pvQkZNQii1,00000533-3f1c-4b3b-81f1-0c8f35754b4e,,,NaN,NaN,NaN,NaN
1,00000716-27e9-4e72-a602-d0ce61784b06,2023-07-18 21:07:24,click_question_open,8QXy31PQxbW9qLzq0Y1dhR8Ypm52,00000716-27e9-4e72-a602-d0ce61784b06,,,64.0,436.0,4830.0,NaN
2,000007c8-68ce-40e6-9b1e-f0e34e8ff9cc,2023-08-06 20:18:03,click_bottom_navigation_profile,6bcea65d-9f40-46fc-888c-700fe707483f,000007c8-68ce-40e6-9b1e-f0e34e8ff9cc,,,26.0,174.0,4729.0,NaN
3,00000981-5e2a-4111-993e-4f1891ad9a53,2023-08-05 01:46:10,view_shop,XVYNT6zfhFWqIg9omwg2AHDjTLx2,00000981-5e2a-4111-993e-4f1891ad9a53,,,61.0,44.0,142.0,NaN
4,00000a7a-ba72-4332-b4a9-7910670aaeb2,2023-07-24 15:03:37,click_bottom_navigation_lab,XFB2SPiGfjbVhvJ3Q3DBsaT3m2B3,00000a7a-ba72-4332-b4a9-7910670aaeb2,,,119.0,545.0,3287.0,NaN



hackle_events_df 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11441319 entries, 0 to 11441318
Data columns (total 11 columns):
 #   Column          Dtype         
---  ------          -----         
 0   event_id        object        
 1   event_datetime  datetime64[ms]
 2   event_key       object        
 3   session_id      object        
 4   id              object        
 5   item_name       object        
 6   page_name       object        
 7   friend_count    float64       
 8   votes_count     float64       
 9   heart_balance   float64       
 10  question_id     float64       
dtypes: datetime64[ms](1), float64(4), object(6)
memory usage: 960.2+ MB

hackle_events_df 통계 요약:


,event_datetime,friend_count,votes_count,heart_balance,question_id
count,11441319,1.068876e+07,1.068676e+07,1.071268e+07,449484.000000
mean,2023-07-29 01:58:17.236000,5.434357e+01,2.572742e+02,1.626929e+04,2766.385262
min,2023-07-18 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,99.000000
25%,2023-07-22 21:31:17,3.200000e+01,9.700000e+01,4.340000e+02,1393.000000
50%,2023-07-28 17:08:58,4.900000e+01,2.100000e+02,1.249000e+03,2569.000000
75%,2023-08-04 17:59:10,7.100000e+01,3.620000e+02,3.188000e+03,4459.000000
max,2023-08-10 23:59:59,1.365000e+03,3.017000e+03,8.849998e+08,5133.000000
std,NaN,3.350798e+01,2.180682e+02,3.317340e+06,1599.967343



hackle_events_df 통계 요약:


,event_id,event_key,session_id,id,item_name,page_name
count,11441319,11441319,11441319,11441319,11441319,11441319
unique,11441319,44,253616,11441319,6,13
top,00000533-3f1c-4b3b-81f1-0c8f35754b4e,view_lab_tap,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,00000533-3f1c-4b3b-81f1-0c8f35754b4e,,
freq,1,1266665,8157,1,11428280,10652540



hackle_events_df 결측치 확인:


event_id                 0
event_datetime           0
event_key                0
session_id               0
id                       0
item_name                0
page_name                0
friend_count        752556
votes_count         754554
heart_balance       728643
question_id       10991835
dtype: int64

In [86]:
# event_id와 id가 같은 건가?
hackle_events_df[hackle_events_df['event_id'] == hackle_events_df['id']].shape 

(11441319, 11)

- event_id는 id와 같음!!

In [87]:
# hackle_properties의 session_id 고유값 개수와 hackle_events의 session_id 고유값 개수가 253616개로 동일함
# session_id 리스트도 같은지 비교!
set(hackle_properties_df['session_id'].unique()) == set(hackle_events_df['session_id'].unique())

True

- hackle_properties의 session_id와 hackle_events의 session_id는 동일함!!

# 4. user_properties

| # |     테이블명    |   컬럼명  |    컬럼 설명   |
|:-:|:---------------:|:---------:|:--------------:|
| 4 | user_properties | class     | 사용자 반      |
|   |                 | gender    | 사용자 성별    |
|   |                 | grade     | 사용자 학년    |
|   |                 | school_id | 사용자 학교 ID |

In [88]:
show_df_info(user_properties_df, 'user_properties_df')


user_properties_df 형태:


(230819, 5)


user_properties_df 첫 5행:


,user_id,class,gender,grade,school_id
0,1000000,1,M,1,1885
1,1000009,10,F,2,3869
2,1000012,10,F,1,5091
3,1000013,8,F,2,1743
4,1000015,2,F,3,5078



user_properties_df 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230819 entries, 0 to 230818
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    230819 non-null  object
 1   class      230819 non-null  int64 
 2   gender     230819 non-null  object
 3   grade      230819 non-null  int64 
 4   school_id  230819 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 8.8+ MB

user_properties_df 통계 요약:


,class,grade,school_id
count,230819.000000,230819.000000,230819.000000
mean,4.594999,2.002197,3083.383335
std,3.151979,0.762048,1711.671162
min,1.000000,1.000000,1.000000
25%,2.000000,1.000000,1594.000000
50%,4.000000,2.000000,3138.000000
75%,6.000000,3.000000,4640.000000
max,20.000000,3.000000,5964.000000



user_properties_df 통계 요약:


,user_id,gender
count,230819,230819
unique,230819,2
top,1000000,F
freq,1,132610



user_properties_df 결측치 확인:


user_id      0
class        0
gender       0
grade        0
school_id    0
dtype: int64

In [90]:
# user_id 컬럼 제외하고 중복 확인 - 없음
user_properties_df.iloc[:,1:].duplicated().sum()

146925

In [91]:
user_properties_df[user_properties_df.iloc[:,1:].duplicated(keep = False)]

,user_id,class,gender,grade,school_id
1,1000009,10,F,2,3869
2,1000012,10,F,1,5091
3,1000013,8,F,2,1743
4,1000015,2,F,3,5078
5,1000018,2,F,3,2523
...,...,...,...,...,...
230813,999990,3,F,2,5104
230814,999992,1,M,2,2240
230816,999997,2,M,3,2502
230817,nhj4wh46MAf5K0IHDu4DGyRsdWn2,5,F,2,3499


In [94]:
user_properties_df[(user_properties_df['school_id'] == 3869) & (user_properties_df['grade'] == 2) &( user_properties_df['class'] == 10)]

,user_id,class,gender,grade,school_id
1,1000009,10,F,2,3869
36059,1100349,10,F,2,3869
41133,1114057,10,F,2,3869
113903,1320610,10,F,2,3869
218883,952243,10,F,2,3869
219168,953354,10,M,2,3869


In [97]:
# user_properties_df의 user_id가 hackle_properties_df의 user_id와 동일한지 확인
set(user_properties_df['user_id'].unique()).issubset(set(hackle_properties_df['user_id'].unique()))


False

In [99]:
# hackle_properties의 user_id에 없는 user_properties의 user_id 확인
not_in_hackle = set(user_properties_df['user_id'].unique()) - set(hackle_properties_df['user_id'].unique())
print("hackle_properties에 없는 user_id 수:", len(not_in_hackle))
print("예시:", list(not_in_hackle)[:5])  # 상위 5개만 보기


hackle_properties에 없는 user_id 수: 1
예시: ['1243402']


In [100]:
hackle_properties_df[hackle_properties_df['user_id'] == '1243402']

,id,session_id,user_id,language,osname,osversion,versionname,device_id
